In [3]:
import pandas as pd
import os
import numpy as np

In [113]:
DATASET_COLUMNS = ['target','ids','date','flag','user','text']

dataset = pd.read_csv("training.1600000.processed.noemoticon.csv", encoding='ISO-8859-1', engine="python", names=DATASET_COLUMNS)
dataset = dataset[['target','text']]
dataset.loc[dataset['target'] == 4, 'target'] = 1
# take the first 10000 rows

# take the last 10000 rows

dataset = pd.concat([df_top, df_bottom])
text = np.array(dataset.text)
target = np.array(dataset.target)
import spacy

nlp = spacy.load('en_core_web_sm')

import re
from spacy.lang.en.stop_words import STOP_WORDS
import string

def preprocess(textdata):
    processedText = np.empty(len(textdata), dtype=object)

    # Defining regex patterns.
    urlPattern = r"((http://)[^ ]*|(https://)[^ ]*|( www\.)[^ ]*)"
    userPattern = '@[^\s]+'
    alphaPattern = "[^a-zA-Z0-9]\s"
    sequencePattern = r"(.)\1\1+"
    seqReplacePattern = r"\1\1"
    wordLessThan2Pattern = r'\b\w{1,2}\b'

    for i, tweet in enumerate(textdata):
        tweet = tweet.lower()

        # Replace all URLs with nothing.
        tweet = re.sub(urlPattern, " ", tweet, flags=re.MULTILINE)
        # Replace @USERNAME with nothing.
        tweet = re.sub(userPattern, " ", tweet)
        # Replace all non-alphabets.
        tweet = re.sub(alphaPattern, " ", tweet)
        # Replace 3 or more consecutive letters by 2 letters.
        tweet = re.sub(sequencePattern, seqReplacePattern, tweet)
        # Remove words less then 2 letters
        tweet = re.sub(wordLessThan2Pattern, '', tweet)
        # Remove punctuation.
        tweet = tweet.translate(str.maketrans("", "", string.punctuation))
        


        # Remove stopwords and lemmatize.
        doc = nlp(tweet)
        filteredWords = [token.lemma_ for token in doc if token.lower_ not in STOP_WORDS]

        # Join the words of a tweet.
        cleanTweet = " ".join(filteredWords)

        # Remove extra whitespace.
        cleanTweet = re.sub(r'\s+', ' ', cleanTweet).strip()

        # Assign the tweet to the processedText array.
        processedText[i] = cleanTweet

    return processedText
processedtext = preprocess(text)
df = pd.DataFrame({ "target":target,"text":processedtext})
processedtextSerie  = pd.Series(processedtext)
emptyRows = df[df.text == ""].index
df.drop(emptyRows,inplace=True)
df[df.text == ""].index
print(len(df))
df.to_csv('cleanData.csv', index=False)


19837


In [5]:
df = pd.read_csv("cleanData.csv")


In [6]:
from collections import Counter

bag_of_words = Counter()
for tweet in df.text:
    words = tweet.split()
    bag_of_words.update(words)

In [7]:
print('word  count')
print('-'*15)
for i, (key, value) in enumerate(bag_of_words.items()):
    if i >= 15:
        break
    print(key, value)

word  count
---------------
aww 176
bummer 22
shoulda 4
get 1145
david 21
carr 2
day 1465
upset 37
update 128
facebook 60
texte 5
cry 98
result 25
school 336
today 1016


In [9]:

from sklearn.feature_extraction.text import TfidfVectorizer
# Create the TF-IDF vectorizer
vectorizer = TfidfVectorizer(token_pattern=r'\b[a-zA-Z]+\b')

# Fit and transform the vectorizer on the tweets
tfidf_matrix = vectorizer.fit_transform(df.text)

# Get the feature names from the vectorizer's vocabulary
feature_names = vectorizer.get_feature_names_out()

# Convert the sparse matrix to a pandas DataFrame
tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=feature_names)

# Print the TF-IDF DataFrame
tfidf_df[:5]

,aack,aaghh,aah,aahh,aahhdont,aaiwa,aaj,aaje,aaliyah,aalsmeer,...,zoo,zoofinally,zoom,zrovna,zumba,zune,zuraidah,zurich,zut,zyrtec
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [102]:
X_train, X_test, y_train, y_test = train_test_split(df['text'], df['target'], test_size=0.2, random_state=42)

# Create TF-IDF vectorizer to convert text data into numerical features
vectorizer = TfidfVectorizer()
X_train_vectorized = vectorizer.fit_transform(X_train)
X_test_vectorized = vectorizer.transform(X_test)

In [105]:
from sklearn.svm import SVC
svm_model = SVC()
svm_model.fit(X_train_vectorized, y_train)
y_pred = svm_model.predict(X_test_vectorized)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.7487399193548387


In [11]:
from sklearn.model_selection import train_test_split

In [12]:
X = df['text']
y = df['target']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(df['text'], df['target'], test_size=0.2, random_state=42)

# Create TF-IDF vectorizer to convert text data into numerical features
vectorizer = TfidfVectorizer()
X_train_vectorized = vectorizer.fit_transform(X_train)
X_test_vectorized = vectorizer.transform(X_test)

X_train = X_train_vectorized.toarray()[0:1000]
X_test = X_test_vectorized.toarray()[0:1000]
y_train = y_train[0:1000]
y_test = y_test[0:1000]

from lazypredict.Supervised import LazyClassifier

clf = LazyClassifier(verbose=0,ignore_warnings=True, custom_metric=None)
models,predictions = clf.fit(X_train, X_test, y_train, y_test)



100%|██████████| 29/29 [04:36<00:00,  9.54s/it]


In [13]:
print(models)

                               Accuracy  Balanced Accuracy  ROC AUC  F1 Score  \
Model                                                                           
ExtraTreesClassifier               0.66               0.66     0.66      0.66   
RandomForestClassifier             0.65               0.65     0.65      0.64   
NearestCentroid                    0.64               0.64     0.64      0.64   
DecisionTreeClassifier             0.64               0.64     0.64      0.63   
XGBClassifier                      0.63               0.64     0.64      0.63   
NuSVC                              0.63               0.63     0.63      0.63   
BaggingClassifier                  0.63               0.63     0.63      0.62   
LogisticRegression                 0.62               0.63     0.63      0.62   
PassiveAggressiveClassifier        0.62               0.62     0.62      0.62   
CalibratedClassifierCV             0.61               0.61     0.61      0.60   
LinearSVC                   